# Implementing polynomial ridge regression

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, get_scorer_names, SCORERS

# Set current working directory
os.chdir('C:/Users/amu_k/subjects-mri-dwi/pre-processing-files/extracted-lab-regions')

# Excel file
excel = pd.ExcelFile('average_values.xlsx')

# Which tensor map-brain region?
df = excel.parse('md-SP')

# Exclude outliers
outliers = [7, 89, 135, 146, 173]
df = df[~df.index.isin(outliers)]

# Initialise X and y
X = df[['ga']].values
y = df[['average']].values

df.head()

In [ ]:
# Define feature and target
X = np.array(df['ga']).reshape(-1,1)
y = np.array(df['average'])


# Define GA ranges
ga_ranges = [
    (20.50, 21.49),
    (21.50, 22.49),
    (22.50, 23.49),
    (23.50, 24.49),
    (24.50, 25.49),
    (25.50, 26.49),
    (26.50, 27.49),
    (27.50, 28.49),
    (28.50, 29.49),
    (29.50, 30.49),
    (30.50, 31.49),
    (31.50, 32.49),
    (32.50, 33.49),
    (33.50, 34.49),
    (34.50, 35.49),
    (35.50, 36.49),
    (36.50, 37.49),
    (37.50, 38.49),
]

# Group the data based on GA ranges
groups = {}
for ga_range in ga_ranges:
    lower, upper = ga_range
    group_name = f"{lower} - {upper}"
    groups[group_name] = df[(df['ga'] >= lower) & (df['ga'] < upper)]

# Initialise empty lists for train and test sets
train_data = []
test_data = []

# Iterate over groups
for group_name, group_data in groups.items():
    # Split group into train and test
    train_size = int(len(group_data) * 0.8)  # 80-2- split
    train_group = group_data[:train_size]
    test_group = group_data[train_size:]
    
    train_data.append(train_group)
    test_data.append(test_group)

# Concatenate train and test data
train_set = pd.concat(train_data)
test_set = pd.concat(test_data)

# Shuffle train and test sets
train_set = train_set.sample(frac=1).reset_index(drop=True)
test_set = test_set.sample(frac=1).reset_index(drop=True)

#print(test_set)

X_train = np.array(train_set['ga']).reshape(-1,1)
y_train = train_set['average']

X_test = np.array(test_set['ga']).reshape(-1,1)
y_test = test_set['average']

#print(X_train.shape)
#print(y_train.shape)

#print(X_test.shape)
#print(y_test.shape)

In [ ]:
#print(list(SCORERS.keys()))

In [ ]:
# Create pipeline
model = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
])

# Define parameter grid for GridSearchCV
param_grid = {
    'poly__degree': [1, 2, 3, 4],  # Polynomial degree
    'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100]  # L2 penalty (lambda)
}

# Initialise GridSearchCV
#grid_search = GridSearchCV(model, param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42), scoring=scoring_metric)

grid_search = GridSearchCV(model, param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42))

# Fit the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
best_degree = best_params['poly__degree']
best_alpha = best_params['ridge__alpha']

# Train the model with the best parameters
#best_model = Pipeline([
    #('poly', PolynomialFeatures(degree=4)),
    #('scaler', StandardScaler()),
    #('ridge', Ridge(alpha=0.01))
#])

# Set model with the best parameters
best_model = grid_search.best_estimator_

#best_model.fit(X, y)

# Evaluate the model
r2_score_train = best_model.score(X_train, y_train)
r2_score_test = best_model.score(X_test, y_test)

# Extract the coefficients
coefficients = best_model.named_steps['ridge'].coef_
intercept = best_model.named_steps['ridge'].intercept_

print("Best Polynomial Degree:", best_degree)
print("Best L2 Penalty (alpha):", best_alpha)
print("Ridge Coefficients:", coefficients)
print("Intercept:", intercept)
print("Model Train R^2 Score:", r2_score_train)
print("Model Test R^2 Score:", r2_score_test)

In [ ]:
# Predict using best model
polyline = np.linspace(20, 39, 200).reshape(-1, 1)
predicted_values = best_model.predict(polyline)

# Plot VM subjects in a different colour
VM_subjects = [3, 8, 53, 54, 137, 166]
normal_subjects = df[~df.index.isin(VM_subjects)]

plt.figure(figsize=(6, 6))
plt.scatter(normal_subjects['ga'], normal_subjects['average'], label='Healthy', color='darkgreen', marker='o', alpha=0.5)
plt.scatter(df['ga'].iloc[VM_subjects], df['average'].iloc[VM_subjects], label='VM', color='crimson', marker='o', alpha=0.8, s=65)
plt.plot(polyline, predicted_values, color='black', label='Polynomial ridge model')
plt.text(0.03, 0.95, f"R²: {r2_score_train:.2f}", transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
plt.text(0.03, 0.89, f"Degree: {best_degree}", transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
plt.text(0.03, 0.83, f"L2 Penalty: {best_alpha}", transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
plt.xlabel('GA (weeks)')
plt.ylabel('Average intensity value')
plt.title('Mean Diffusivity Subplate')

# Set integer ticks for x-axis
plt.xticks(np.arange(20, 40, 2))

plt.grid(True)
plt.legend(loc='upper right', fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# Reshape polyline to have two dimensions
polyline = np.linspace(20, 39, 200).reshape(-1,1)

# Calculate the second derivative of the polynomial function
second_derivative = derivative(best_model.predict, polyline, n=2)

# Find where the sign changes in the second derivative
inflection_indices = np.where(np.diff(np.sign(second_derivative)))[0]

# Check if these potential inflection points are actual inflection points
for idx in inflection_indices:
    inflection_point_x = polyline[idx]
    print("Inflection point x-coordinate:", inflection_point_x)

In [ ]:
# Define variable names
x = 'ga'
y = 'avg'

# Create empty equation starting with intercept
equation = f"{y} = {intercept:.5f}"

# Add terms depending on polynomial degree
if best_degree == 2:
    # For 2nd degree polynomial
    equation += f"{coefficients[1]:+.5f}*{x}{coefficients[2]:+.5f}*{x}²"
elif best_degree == 3:
    # For 3rd degree polynomial
    equation += f"{coefficients[1]:+.5f}*{x}{coefficients[2]:+.5f}*{x}²{coefficients[3]:+.5f}*{x}³"

print(equation)

In [ ]:
# Define variable names
#x = 'ga'
#y = 'avg'

# Create empty equation starting with intercept
#equation = f"{y} = {intercept:.5f}"

# Add terms depending on polynomial degree
#if best_degree == 2:
    # For 2nd degree polynomial
    #equation += f"{coefficients[1]:+.5f}*{x}{coefficients[2]:+.5f}*{x}²"
#elif best_degree == 3:
    # For 3rd degree polynomial
    #equation += f"{coefficients[1]:+.5f}*{x}{coefficients[2]:+.5f}*{x}²{coefficients[3]:+.5f}*{x}³"

#print(equation)

In [ ]:
# Extract coefficients and intercept from best model
#best_coefficients = best_model.named_steps['ridge'].coef_
#best_intercept = best_model.named_steps['ridge'].intercept_

# Get best polynomial degree
#best_degree = best_params['poly__degree']

# Define variable names
x = 'ga'
y = 'avg'

# Create empty equation starting with intercept
equation = f"{y} = {intercept:.5f}"

# Add terms depending on polynomial degree
for i in range(1, best_degree + 1):
  coefficient = coefficients[i-1]
  if coefficient != 0:  # Only add terms with non-zero coefficients
    if coefficient > 0:
      sign ='+'
    else:
      sign ='-'
    equation += f"{sign}{abs(coefficient):.5f}*{x}^{i}"

# Print equation
print("Equation of the fitted model:")
print(equation)